***Vorlesung 'Syntax natürlicher Sprachen'***

--- 
# Intro Vorlesung 4: Phrasenstrukturgrammatiken


## 0. Informationen zur Phrasenstruktur im Deutschen


- Übersicht Phrasen: https://grammis.ids-mannheim.de/progr@mm/5213
- Nominalphrase: https://grammis.ids-mannheim.de/progr@mm/5302
- Verbalphrase: https://grammis.ids-mannheim.de/terminologie/279
- Präpositionalphrase: https://grammis.ids-mannheim.de/progr@mm/5286
- Adjektivphrase: https://grammis.ids-mannheim.de/progr@mm/1644
- Adverbphrase: https://grammis.ids-mannheim.de/progr@mm/1653


## 1. vom Konstituententest zu Phrasenstrukturregeln


---
### 1.1 CFG mit flacher Strukturregel (POS-Muster) für Nominalphrase NP

In [1]:
import nltk

In [2]:
grammar = nltk.CFG.fromstring("""
    NP -> DET PRT ADJ N
    
    DET -> 'die'
    N -> 'Katze'
    ADJ -> 'große' | 'schlaue'
    PRT -> 'sehr'
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'die sehr große Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

     NP             
 ┌───┼─────┬─────┐   
DET PRT   ADJ    N  
 │   │     │     │   
die sehr große Katze



---
#### + Eliminierung *sehr*

In [3]:
#wohlgeformt:
sent = 'die große Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)
    #kein Parse (> Regel hinzufügen, da wohlgeformt)

In [4]:
# neue Grammatikregel:
grammar = nltk.CFG.fromstring("""
    NP -> DET PRT ADJ N
    NP -> DET ADJ N

    DET -> 'die'
    N -> 'Katze'
    ADJ -> 'große' | 'schlaue'
    PRT -> 'sehr'
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'die große Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

      NP       
 ┌────┼─────┐   
DET  ADJ    N  
 │    │     │   
die große Katze



---
#### - Eliminierung *große*

In [5]:
#nicht wohlgeformt:
sent = 'die sehr Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)
    #kein Parse (> keine Regel NP -> DET PRT N hinzu, da nicht wohlgeformt!)

---
#### + Eliminierung *sehr große*



In [6]:
#wohlgeformt:
sent = 'die Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)
    #kein Parse (> Regel hinzufügen, da wohlgeformt)

In [7]:
# neue Regel:
grammar = nltk.CFG.fromstring("""
    NP -> DET PRT ADJ N
    NP -> DET ADJ N
    NP -> DET N

    DET -> 'die'
    N -> 'Katze'
    ADJ -> 'große' | 'schlaue'
    PRT -> 'sehr'
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'die Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

     NP      
 ┌───┴────┐   
DET       N  
 │        │   
die     Katze



---
### 1.2 CFG als hierarchische Phrasenstrukturgrammatik (mit ADJP  = Adjektivphrase):

#### festgestellte Regeln:
```
        NP -> DET PRT ADJ N
        NP -> DET ADJ N
        NP -> DET N
```
#### nicht möglich:
```
        *NP -> DET PRT N 
```
#### Beobachtung:

- PRT kann nur zusammen mit ADJ auftreten
- ADJ kann ohne PRT auftreten und ist dann eliminierbar
- PRT und ADJ bilden optionale Einheit in NP, mit dem ADJ als Kopf = **ADJP (Adjektivphrase)**


In [8]:
grammar = nltk.CFG.fromstring("""
    NP -> DET ADJP N
    
    ADJP -> PRT ADJ | ADJ
    
    NP -> DET N
    
    DET -> 'die'
    N -> 'Katze'
    ADJ -> 'große' | 'schlaue'
    PRT -> 'sehr'
""")

parser = nltk.ChartParser(grammar,trace=0)

In [9]:
sent = 'die sehr große Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

     NP                  
 ┌───┴────┬───────────┐   
 │       ADJP         │  
 │   ┌────┴─────┐     │   
DET PRT        ADJ    N  
 │   │          │     │   
die sehr      große Katze



In [10]:
sent = 'die große Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

      NP       
 ┌────┼─────┐   
 │   ADJP   │  
 │    │     │   
DET  ADJ    N  
 │    │     │   
die große Katze



In [11]:
sent = 'die Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

     NP      
 ┌───┴────┐   
DET       N  
 │        │   
die     Katze



In [12]:
#nicht wohlgeformt:
sent = 'die sehr Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)
    #kein Parse

---
### 1.3 CFG mit X-Bar-Struktur: ADJP als rekursives Adjunkt

- NOM als Zwischenebene (= *sehr große Katze*)

In [13]:
# bisher: keine Regel für rekursive ADJP-Adjunktion
sent = 'die sehr große sehr schlaue Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)
    #kein Parse, da keine Regel

In [14]:
grammar = nltk.CFG.fromstring("""
    NP -> DET NOM
    
    NOM -> ADJP NOM
    NOM -> N
    
    ADJP -> PRT ADJ | ADJ


    DET -> 'die'
    N -> 'Katze'
    ADJ -> 'große' | 'schlaue'
    PRT -> 'sehr'
""")

parser = nltk.ChartParser(grammar,trace=0)

---
#### Die rekursive Regel `NOM -> ADJP NOM` erlaubt rekursive Adjunktion von ADJPs:

In [15]:
sent = 'die sehr große schlaue Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

                NP                       
 ┌──────────────┴────┐                    
 │                  NOM                  
 │        ┌──────────┴───────────┐        
 │        │                     NOM      
 │        │                ┌─────┴────┐   
 │       ADJP             ADJP       NOM 
 │   ┌────┴─────┐          │          │   
DET PRT        ADJ        ADJ         N  
 │   │          │          │          │   
die sehr      große     schlaue     Katze



In [16]:
sent = 'die sehr große Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

          NP             
 ┌────────┴─────┐         
 │             NOM       
 │        ┌─────┴─────┐   
 │       ADJP        NOM 
 │   ┌────┴─────┐     │   
DET PRT        ADJ    N  
 │   │          │     │   
die sehr      große Katze



---
#### gleichzeitig wird durch die Regel `NOM -> ADJP NOM` mit NOM-Zwischenebene die Erkennung (bzw. Übergenerierung) nicht-grammatischer NP-Strukturen verhindert (im Gegensatz zur Regel `NP -> ADJP NP`):

In [17]:
#nicht wohlgeformt:
sent = 'die sehr große die Katze'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)
#kein Parse

### 1.4 Erweiterung um X-Bar-Komplement (Genitiv-NPs)

In [18]:
grammar = nltk.CFG.fromstring("""
    NP -> DET NOM
    
    NOM -> ADJP NOM
    NOM -> N
    
    NOM -> N NP
    
    ADJP -> PRT ADJ | ADJ


    DET -> 'die' | 'der' | 'des'
    N -> 'Katze' | 'Besitzerin' | 'Hauses'
    ADJ -> 'große' | 'schlaue'
    PRT -> 'sehr'
""")

parser = nltk.ChartParser(grammar,trace=0)

---
#### Die Regel `NOM -> N NP` erlaubt Hinzufügen von Genitiv-NPs als Komplementen zum Kopf (indirekt rekursiv, da auch `NP -> DET NOM`):

In [19]:
sent = 'die sehr große schlaue Katze der Besitzerin des Hauses'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)

                NP                                                 
 ┌──────────────┴─────────────┐                                     
 │                           NOM                                   
 │        ┌───────────────────┴────┐                                
 │        │                       NOM                              
 │        │            ┌───────────┴──────┐                         
 │        │            │                 NOM                       
 │        │            │      ┌───────────┴───────┐                 
 │        │            │      │                   NP               
 │        │            │      │    ┌──────────────┴───┐             
 │        │            │      │    │                 NOM           
 │        │            │      │    │      ┌───────────┴───┐         
 │        │            │      │    │      │               NP       
 │        │            │      │    │      │           ┌───┴────┐    
 │       ADJP         ADJP    │    │     


---
## 2. Umwandlung von CFG mit flachen Regeln in  X-Bar-Struktur


In [20]:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
    VP  -> V NP
 
    NP  -> DET N
    NP  -> DET ADJ N

    DET -> "Der" | "den"
    N   -> "Hund" | "Briefträger"
    ADJ -> "langsamen"
    V   -> "jagt"
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'Der Hund jagt den langsamen Briefträger'.split()

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

              S                                
     ┌────────┴────────┐                        
     │                 VP                      
     │        ┌────────┴──────┐                 
     NP       │               NP               
 ┌───┴───┐    │    ┌──────────┼──────────┐      
DET      N    V   DET        ADJ         N     
 │       │    │    │          │          │      
Der     Hund jagt den     langsamen Briefträger



---

### 2.1 NP-Adjunkt (ADJ) und NP-Spezifizierer (DET)

- **Umwandlung in X-Bar-Struktur, um rekursive Erweiterung um Adjektive zu ermöglichen:**

*Der Hund jagt den **langsamen schreienden** Briefträger*

- **Gleichzeitig sollen Überproduktionen folgender Art vermieden werden:**

\* *Der Hund jagt den langsamen schreienden **den** Briefträger*


In [21]:
#X-Bar mit NP-Adjunkt (ADJ):
grammar = nltk.CFG.fromstring("""
    S  -> NP VP
   
    VP -> V NP
    
#NP-SPEZIFIZIERER:    
    NP  -> DET NOM
#NP-ADJUNKT:
    NOM -> ADJ NOM
#NP-KOMPLEMENT-REGEL (ohne Komplement, nur Kopf):
    NOM -> N
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger"
    ADJ -> "langsamen" | "schreienden"
    V   -> "jagt"
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'der Hund jagt den langsamen schreienden Briefträger'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

                   S                                           
     ┌─────────────┴──────┐                                     
     │                    VP                                   
     │        ┌───────────┴──────┐                              
     │        │                  NP                            
     │        │    ┌─────────────┴───────┐                      
     │        │    │                    NOM                    
     │        │    │      ┌──────────────┴───────┐              
     NP       │    │      │                     NOM            
 ┌───┴───┐    │    │      │              ┌───────┴───────┐      
 │      NOM   │    │      │              │              NOM    
 │       │    │    │      │              │               │      
DET      N    V   DET    ADJ            ADJ              N     
 │       │    │    │      │              │               │      
der     Hund jagt den langsamen     schreienden     Briefträger



In [22]:
#Negativ-Beispiel mit wiederholtem Determinierer (wird nicht erkannt):
sent = 'der Hund jagt den langsamen schreienden den Briefträger'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)   
    #kein Parse

---
### 2.2 NP-Komplement (Genitiv-NP) und VP-Komplement (Objekt-NP)

In [23]:
#X-Bar mit NP-Komplement (Genititv-NP):
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
#VP-SPEZIFIZIERER (ohne):   
    VP  -> VERBAL 
#VP-KOMPLEMENT (transitives Verb):    
    VERBAL -> V NP
        
    NP  -> DET NOM
    NOM -> ADJ NOM
    NOM -> N
#neue NP-KOMPLEMENT-Regel (Genititv-NP):    
    NOM -> N NP
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger" | "Stadt"
    ADJ -> "langsamen" | "schreienden"
    V   -> "jagt"
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'der Hund jagt den Briefträger der Stadt'.split()
for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

                   S                               
     ┌─────────────┴───────┐                        
     │                     VP                      
     │                     │                        
     │                   VERBAL                    
     │        ┌────────────┴───────┐                
     │        │                    NP              
     │        │    ┌───────────────┴───┐            
     │        │    │                  NOM          
     │        │    │       ┌───────────┴───┐        
     NP       │    │       │               NP      
 ┌───┴───┐    │    │       │           ┌───┴────┐   
 │      NOM   │    │       │           │       NOM 
 │       │    │    │       │           │        │   
DET      N    V   DET      N          DET       N  
 │       │    │    │       │           │        │   
der     Hund jagt den Briefträger     der     Stadt



In [24]:
#Beispiel mit verschachtelten Genitiv-NPs:
sent = 'der Hund jagt den Briefträger der Stadt der Briefträger'.split()

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

         S                                                            
     ┌───┴──────────┐                                                  
     │              VP                                                
     │              │                                                  
     │            VERBAL                                              
     │        ┌─────┴─────────────────┐                                
     │        │                       NP                              
     │        │     ┌─────────────────┴────┐                           
     │        │     │                     NOM                         
     │        │     │         ┌────────────┴────┐                      
     │        │     │         │                 NP                    
     │        │     │         │       ┌─────────┴───┐                  
     │        │     │         │       │            NOM                
     │        │     │         │       │    ┌────────┴───┐              

---
### 2.3 mehr VP-Komplemente (Objekt und indirektes Objekt)

- 2 Analysen (Ambiguität):
   - *der*: Genitiv (NP-Komplement) oder Dativ (VP-Komplement)!
   - beide Analysen linguistisch möglich (*übergeben* kann sowohl ditransitiv als auch transitiv verwendet werden)

In [25]:
#X-Bar:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
    VP  -> VERBAL 
    VERBAL -> V NP
#neue VP-KOMPLEMENT-Regel (ditransitives Verb: 2 Komplemente):        
    VERBAL -> V NP NP


    NP  -> DET NOM
    NOM -> ADJ NOM
    NOM -> N
    NOM -> N NP
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger" | "Stadt"
    ADJ -> "langsamen" | "schreienden"
    V   -> "jagt" | "übergibt"
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'der Briefträger übergibt den Hund der Stadt'.split()

print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

2
                              S                          
     ┌────────────────────────┴────────┐                  
     │                                 VP                
     │                                 │                  
     │                               VERBAL              
     │                 ┌──────────┬────┴─────────┐        
     NP                │          NP             NP      
 ┌───┴───────┐         │      ┌───┴────┐     ┌───┴────┐   
 │          NOM        │      │       NOM    │       NOM 
 │           │         │      │        │     │        │   
DET          N         V     DET       N    DET       N  
 │           │         │      │        │     │        │   
der     Briefträger übergibt den      Hund  der     Stadt

                              S                          
     ┌────────────────────────┴────┐                      
     │                             VP                    
     │                             │                      
   

---
### 2.4 VP-Spezifizierer (AUX) mit Zusatzregel für invertiertes Komplement:


In [26]:
#X-Bar (ausgehend von Grammatik ohne IV/TV/DTV-Differenzierung):
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    
    VP  -> VERBAL 
    VERBAL -> V NP
    VERBAL -> V NP NP
#neuer VP-SPEZIFIZIERER (Auxiliar):        
    VP -> AUX VERBAL
#neue VP-KOMPLEMENT-REGEL mit invertierter Wortstellung: 
    VERBAL -> NP V

    NP  -> DET NOM
    NOM -> ADJ NOM
    NOM -> N
    NOM -> N NP
 
    DET -> "der" | "den"
    N   -> "Hund" | "Briefträger" | "Stadt"
    ADJ -> "langsamen" | "schreienden" | "ganzen"
    V   -> "jagt" | "übergibt"
#neue lexikalische Regeln für AUX:
    AUX -> "hat"
    V -> "übergeben"
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'der Briefträger hat den Hund übergeben'.split()

print(len(list(parser.parse(sent))))

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

1
                     S                          
     ┌───────────────┴───────┐                   
     │                       VP                 
     │               ┌───────┴────┐              
     │               │          VERBAL          
     │               │       ┌────┴────────┐     
     NP              │       NP            │    
 ┌───┴───────┐       │   ┌───┴────┐        │     
 │          NOM      │   │       NOM       │    
 │           │       │   │        │        │     
DET          N      AUX DET       N        V    
 │           │       │   │        │        │     
der     Briefträger hat den      Hund  übergeben

